## Finding the Best Way to Swap Coins


In [1]:
import pandas as pd
from IPython import display
from IPython.core.display import HTML

from src.data_extractors import get_pools, get_prices
from config import BOSTROM_NODE_URL, POOL_FEE


source_coin = "boot"
target_coin = "tocyb"
source_amount = 89_200_000_000
max_slippage = 0.15
min_path_decrease_premium = 0.005


In [2]:
import os 
path = "/Users/user/Documents/local_files/cyber_files/"
os.chdir(path)

#### Get Bostrom and Osmosis Pools Data


In [3]:
pools_df = get_pools(network="bostrom", display_data=True)


network,id,type_id,balances,swap_fee,reserve_coin_denoms
bostrom,1,1,"[{'denom': 'boot', 'amount': '1149315316171'}, ...",0.003,"[boot, hydrogen]"
bostrom,2,1,"[{'denom': 'boot', 'amount': '13887244460'}, {'...",0.003,"[boot, milliampere]"
bostrom,3,1,"[{'denom': 'boot', 'amount': '1139152020166'}, ...",0.003,"[boot, tocyb]"
bostrom,4,1,"[{'denom': 'hydrogen', 'amount': '2013603314650...",0.003,"[hydrogen, tocyb]"
bostrom,5,1,"[{'denom': 'hydrogen', 'amount': '264381074900'...",0.003,"[hydrogen, milliampere]"
bostrom,6,1,"[{'denom': 'hydrogen', 'amount': '395841379406'...",0.003,"[hydrogen, millivolt]"
bostrom,7,1,"[{'denom': 'hydrogen', 'amount': '149542496640'...",0.003,"[hydrogen, uosmo in bostrom]"
bostrom,8,1,"[{'denom': 'hydrogen', 'amount': '20967017356'}...",0.003,"[hydrogen, uatom in bostrom]"
bostrom,9,1,"[{'denom': 'uosmo in bostrom', 'amount': '15754...",0.003,"[uosmo in bostrom, uatom in bostrom]"
bostrom,10,1,"[{'denom': 'milliampere', 'amount': '25235366'}...",0.003,"[milliampere, millivolt]"


In [4]:
# Импорт csv - перенес в cyberarbitrage потому что там есть осмо

# import datetime

# _list = pools_df["balances"].to_list()
# _t = []
# for i in _list:
#     _t.append([i[0]["denom"], i[0]["amount"], i[1]["denom"], i[1]["amount"]])

# _pools = pd.DataFrame.from_records(
#     _t,
#     columns=["Coin 1", "Coin 1 pool amount", "Coin 2", "Coin 2 pool amount"],
# )

# _pools['update_time'] = datetime.datetime.now()

# _pools.to_csv("/Users/user/Documents/!temp/Excel_folder/cyber_files/pools_log.csv" , mode='a', index=False, header=False)

#### Calculate Prices


In [ ]:
price_df = get_prices(pools_df=pools_df, display_data=True)


#### Search the best way to swap coins


In [ ]:
# main: coin_source -> coin_target
ways = []
pool_main_df = pools_df[
    (pools_df.reserve_coin_denoms.isin([[source_coin, target_coin]]))
    | (pools_df.reserve_coin_denoms.isin([[target_coin, source_coin]]))
]
if len(pool_main_df) > 0:
    ways = [[[source_coin, target_coin]]]

# alternative: coin_source -> coin3 -> coin_target
coin3_list = list(
    price_df[
        (~price_df[source_coin].isna())
        & (~price_df[target_coin].isna())
        & (~price_df.index.isin([source_coin, target_coin]))
    ].index
)
for coin3 in coin3_list:
    ways.append([[source_coin, coin3], [coin3, target_coin]])


def way_to_str(way: list):
    return ", ".join(f"{way_item[0]} -> {way_item[1]}" for way_item in way)


print(f"ways count {len(ways)}")
print("".join(f"{str(i + 1)}. {way_to_str(way)}\n" for i, way in enumerate(ways)))


In [ ]:
def get_balance_by_coin(pool_balances: list, coin: str) -> int:
    try:
        return [
            float(item["amount"]) for item in pool_balances if item["denom"] == coin
        ][0]
    except Exception as e:
        print(pool_balances, coin, e)


def generate_swap_query(
    coin_from_amount: float,
    coin_from: str,
    coin_to: str,
    coins_pool_df: pd.DataFrame,
    price_df: pd.DataFrame = price_df,
    max_slippage: float = max_slippage,
    wallet: str = "$WALLET",
    chain_id: str = "bostrom",
    node=BOSTROM_NODE_URL,
) -> str:

    _pool_id = coins_pool_df.loc[:, "id"].to_list()[0]
    _pool_type = coins_pool_df.loc[:, "type_id"].to_list()[0]
    _price = price_df.loc[coin_from, coin_to] * (1 + max_slippage)
    return (
        f"cyber tx liquidity swap {_pool_id} {_pool_type} {int(coin_from_amount)}{coin_from} {coin_to} {_price:.6f} 0.003 --from {wallet} "
        f"--chain-id {chain_id} --gas 200000 --gas-prices 0.01boot --yes --node {node} --broadcast-mode block"
    )


def calculate_swap(
    way: list, coin1_amount: float, pools_df: pd.DataFrame = pools_df
) -> [float, list]:
    _coin_from_amount = coin1_amount
    coin2_way_queries = []
    for way_item in way:
        _coin_from = way_item[0]
        _coin_to = way_item[1]
        _coins_pool_df = pools_df[
            (pools_df.reserve_coin_denoms.isin([[_coin_from, _coin_to]]))
            | (pools_df.reserve_coin_denoms.isin([[_coin_to, _coin_from]]))
        ]
        _coin_from_pool_amount = get_balance_by_coin(
            _coins_pool_df.balances.values[0], _coin_from
        )
        _coin_to_pool_amount = get_balance_by_coin(
            _coins_pool_df.balances.values[0], _coin_to
        )
        _coin_to_amount = (
            _coin_from_amount
            * _coin_to_pool_amount
            / (_coin_from_pool_amount + 2 * _coin_from_amount)
            * (1 - POOL_FEE)
        )
        coin2_way_queries.append(
            generate_swap_query(
                coin_from_amount=_coin_from_amount,
                coin_from=_coin_from,
                coin_to=_coin_to,
                coins_pool_df=_coins_pool_df,
            )
        )
        _coin_from_amount = _coin_to_amount
    coin2_way_amount = _coin_from_amount
    return coin2_way_amount, coin2_way_queries


def calculate_all_swap(
    amount_ways_list: list, print_message: bool = False
) -> [float, list]:
    swap_amount = 0
    swap_queries = []
    for _way, _amount in amount_ways_list:
        _swap_amount_way, _swap_queries_way = calculate_swap(_way, coin1_amount=_amount)
        swap_amount += _swap_amount_way
        swap_queries.append(_swap_queries_way)
        if print_message:
            print(f"{_way}\n{_swap_amount_way}\n")
    return swap_amount, swap_queries


def get_best_swap(
    ways: list,
    amount=source_amount,
    number_points: int = 10,
    min_path_decrease_premium: float = min_path_decrease_premium,
    print_message: bool = False,
) -> [pd.DataFrame, float]:
    if len(ways) == 0:
        return pd.DataFrame(columns=["way", "amount", "swap_queries"]), 0.0
    elif len(ways) == 1:
        _amount_list = [amount]
    elif len(ways) == 2:
        _amount_list = [
            [
                coef_1 / number_points * amount,
                (number_points - coef_1) / number_points * amount,
            ]
            for coef_1 in range(number_points + 1)
        ]
    elif len(ways) == 3:
        _amount_list = [
            [
                coef_1 / number_points * amount,
                coef_2 / number_points * amount,
                (number_points - coef_1 - coef_2) / number_points * amount,
            ]
            for coef_1 in range(number_points + 1)
            for coef_2 in range(number_points - coef_1 + 1)
        ]
    else:
        _amount_list = [
            [
                coef_1 / number_points * amount,
                coef_2 / number_points * amount,
                coef_3 / number_points * amount,
                (number_points - coef_1 - coef_2 - coef_3) / number_points * amount,
            ]
            for coef_1 in range(number_points + 1)
            for coef_2 in range(number_points - coef_1 + 1)
            for coef_3 in range(number_points - coef_1 - coef_2 + 1)
        ]

    _swap_amount_list = []
    for _amount_list_item in _amount_list:
        _amount_ways_list = [
            [_way, _amount] for _way, _amount in zip(ways, _amount_list_item)
        ]
        _swap_result, _swap_queries = calculate_all_swap(_amount_ways_list)
        _swap_amount_list.append([_amount_list_item, _swap_result, _swap_queries])

    _swap_amount_df = pd.DataFrame(
        _swap_amount_list, columns=["amount_by_way_item", "swap_result", "swap_queries"]
    ).sort_values("swap_result", ascending=False)
    _swap_amount_df["effective_swap_result"] = _swap_amount_df.apply(
        lambda x: x["swap_result"]
        * (
            1
            - min_path_decrease_premium
            * len([item for item in x["amount_by_way_item"] if item != 0])
        ),
        axis=1,
    )
    if print_message:
        display(
            HTML(
                (
                    _swap_amount_df.to_html(
                        index=False, notebook=True, show_dimensions=False
                    )
                )
            )
        )

    _swap_max_amount_list = list(
        _swap_amount_df.loc[_swap_amount_df["effective_swap_result"].idxmax()][
            ["amount_by_way_item", "swap_queries"]
        ]
    )
    return (
        pd.DataFrame(
            zip(ways, _swap_max_amount_list[0], _swap_max_amount_list[1]),
            columns=["way", "amount", "swap_queries"],
        ),
        _swap_amount_df["swap_result"].max(),
    )


In [ ]:
best_swap_df, max_result = get_best_swap(ways=ways, number_points=10)
display(HTML(best_swap_df.to_html(index=False, notebook=True, show_dimensions=False)))
print(
    f"Source amount {source_amount:>,} {source_coin}\nMax target amount {int(max_result):>,} {target_coin}\n"
)

for index, row in best_swap_df.iterrows():
    if row.amount > 0:
        print(f"{way_to_str(row.way)}")
        print(f"Amount {int(row.amount):>,} {source_coin}")
        print("\n".join(row.swap_queries), "\n")
